In [11]:
import sys
import ifcopenshell
import math

In [18]:
m=ifcopenshell.open(r"C:\Users\87457\Desktop\IFC2x3 2.0.ifc")
elements=m.by_type("IfcBuildingElement")
elements
len(elements)

214

In [ ]:
"""
e_test=[]
for e in elements:
    if e.is_a("IfcReinforcingBar"):
        e_test.append(e)
e_test
len(e_test)
"""

确定ifc中entity的类型有几种，不是objecttype

In [13]:
elements_entity_type=[]

for e in elements:
    e_entity_type=e.is_a()
    if e_entity_type not in elements_entity_type:
        elements_entity_type.append(e_entity_type)

elements_entity_type

['IfcBuildingElementProxy',
 'IfcWall',
 'IfcWallStandardCase',
 'IfcDoor',
 'IfcStair',
 'IfcStairFlight',
 'IfcMember',
 'IfcWindow',
 'IfcSlab',
 'IfcColumn',
 'IfcBeam',
 'IfcRailing']

确定有几种objecttype以及它们的数量

In [14]:
#用字典来筛选objecttype和number
elements_type_number={}
for e in elements:
    i=1

    e_type=e.ObjectType
    #将新查询到的objecttype与已经查询出来的objecttype对比
    if elements_type_number.get(e_type)==None:
        elements_type_number[e_type]=i
    elif elements_type_number.get(e_type)!=None:
        j=elements_type_number[e_type]
        elements_type_number[e_type]=j+1

elements_type_number
#len(elements_type_number)

{'屋顶板3.0:屋顶板3.0': 1,
 '地面板:地面板': 1,
 'YTTM Lower Level:YTTM Lower Level': 1,
 '基本墙:常规 - 200mm': 25,
 '基本墙:常规 - 225mm 砌体': 10,
 '旋转门:2400 x 2500mm': 1,
 '组合楼梯:190mm 最大踢面 250mm 梯段': 4,
 '非整体梯段:50mm 踏板 13mm 踢面': 4,
 '梯边梁:梯边梁 - 50mm 宽度': 8,
 '玻璃隔断BG2722-BG2822:BG2022_B 3000': 9,
 '玻璃隔断:1000*4000mm': 2,
 '玻璃隔断:1000*3500mm': 2,
 '玻璃隔断:1000*3000mm': 2,
 '玻璃隔断:2000*2500': 2,
 "门联窗_四扇:A_门联窗_四扇01_MLC4526'": 2,
 '玻璃雨篷:食堂玻璃雨蓬': 16,
 '玻璃隔断BG2722-BG2822:BG2022_B 3000*1000': 7,
 '基本墙:内部 - 砌块墙 100': 49,
 '基本墙:常规 - 140mm 砌体': 7,
 '单扇 - 与墙齐:750 x 2000mm': 5,
 '楼板:常规 - 120mm': 2,
 '混凝土 - 圆形 - 柱:1000': 25,
 '混凝土 - 矩形梁:1000 x 800mm': 4,
 '混凝土 - 矩形梁:500 x 800mm': 10,
 '固定:0406 x 0610mm': 1,
 '栏杆扶手:900mm 圆管': 13,
 '栏杆扶手:玻璃嵌板 - 底部填充': 1}

构件体积

Test:
volume existed: column/beam/wall/IfcBuildingElementProxy/IfcSlab
volume lost: IfcReinforcingBar-“体积”这个量存在的位置和其他构件是不同的

In [34]:
"""
e_test=[]
for e in elements:
    if e.is_a("IfcRailing"):
        e_test.append(e)
e_test
#len(e_test)
"""


'\ne_test=[]\nfor e in elements:\n    if e.is_a("IfcRailing"):\n        e_test.append(e)\ne_test\n#len(e_test)\n'

In [35]:
volumes=[]
for e in elements:
    if e.is_a("IfcReinforcingBar"):
        e_props = e.IsDefinedBy
        for e_prop in e_props:
            if e_prop.is_a("IfcRelDefinesByProperties"):
                e_prop_set = e_prop.RelatingPropertyDefinition
                if e_prop_set.Name == "结构" or e_prop_set.Name == "Structure":
                    e_set_value = e_prop_set.HasProperties
                    for e_value in e_set_value:
                        if e_value.Name == "钢筋体积" or e_value.Name == "Volume": #英文待确定
                            v = e_value.NominalValue
                            volumes.append(v.wrappedValue)
    else:
        e_props = e.IsDefinedBy
        for e_prop in e_props:
            if e_prop.is_a("IfcRelDefinesByProperties"):
                e_prop_set = e_prop.RelatingPropertyDefinition
                if e_prop_set.Name == "尺寸标注" or e_prop_set.Name == "Dimensions":
                    e_set_value = e_prop_set.HasProperties
                    for e_value in e_set_value:
                        if e_value.Name == "体积" or e_value.Name == "Volume": #英文待确定
                            v = e_value.NominalValue
                            volumes.append(v.wrappedValue)
                    
volumes
len(volumes)


184

构件材料

Test:
Volume Existed: Column/Beam/Wall/IfcSlab/IfcBuildingElementProxy

In [17]:
mas=[]
for e in elements:
    if e.HasAssociations!=():
        e_ass=e.HasAssociations
        e_ass=e_ass[0]
        e_ma=e_ass.RelatingMaterial
        mas.append(e_ma)
    elif e.HasAssociations==():
        mas.append("null")

mas
#len(mas)

mas_final=[]
for ma in mas:
    if ma=="null":
        ma=ma
    elif ma.is_a("IfcMaterial"):
        ma=ma.Name
    elif ma.is_a("IfcMaterialList"):
        ma=ma.Materials #ma.Material是个元组
    elif ma.is_a("IfcMaterialLayerSetUsage"): #针对墙
        ma_set=ma.ForLayerSet
        ma_setlayers=ma_set.MaterialLayers[0]
        ma=ma_setlayers.Material
        ma=ma.Name #如果没有Name，可删
    mas_final.append(ma)

mas_final 
len(mas_final)

214